***
AI-Powered Film Editing Tool
Automates video editing tasks including scene detection, beat synchronization,
color grading, noise removal, and intelligent clip arrangement.
***

Import all necessary libraries

In [ ]:
import os
import numpy as np
from moviepy.editor import *
from moviepy.video.fx import all as vfx
import librosa
import cv2
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

***
Creating Class for processing and editing video

- detect_scenes: Detect scene changes using frame difference analysis
- detect_beats: Detect musical beats in audio for rhythm-based cutting
- auto_color_grade: Apply AI-based color grading
- create_beat_sync_edit: Create a music video style edit synced to beats
- stabilize_video: Apply basic video stabilization
- enhance_resolution: Upscale video resolution
- export: Export the final video

In [ ]:


class AIFilmEditor:
    def __init__(self, input_video_path, output_path="edited_video.mp4"):
        """
        Initialize the AI Film Editor
        
        Args:
            input_video_path: Path to raw video file
            output_path: Path for output edited video
        """
        self.input_path = input_video_path
        self.output_path = output_path
        self.video = VideoFileClip(input_video_path)
        self.audio = self.video.audio
        self.scenes = []
        self.beat_times = []
        
    def detect_scenes(self, threshold=30.0):
        """
        Detect scene changes using frame difference analysis
        
        Args:
            threshold: Sensitivity for scene detection (lower = more scenes)
        """
        print("🎬 Detecting scenes...")
        prev_frame = None
        scene_timestamps = [0]
        
        for i, frame in enumerate(self.video.iter_frames(fps=5)):
            if prev_frame is not None:
                # Calculate frame difference
                diff = np.mean(np.abs(frame.astype(float) - prev_frame.astype(float)))
                
                if diff > threshold:
                    timestamp = i / 5.0  # Convert frame index to time
                    scene_timestamps.append(timestamp)
            
            prev_frame = frame
        
        scene_timestamps.append(self.video.duration)
        
        # Create scene clips
        for i in range(len(scene_timestamps) - 1):
            start = scene_timestamps[i]
            end = scene_timestamps[i + 1]
            if end - start > 0.5:  # Minimum scene length
                self.scenes.append((start, end))
        
        print(f"✅ Detected {len(self.scenes)} scenes")
        return self.scenes
    
    def detect_beats(self, sensitivity=0.3):
        """
        Detect musical beats in audio for rhythm-based cutting
        
        Args:
            sensitivity: Beat detection sensitivity (0-1)
        """
        print("🎵 Analyzing audio beats...")
        
        # Extract audio
        audio_path = "temp_audio.wav"
        self.video.audio.write_audiofile(audio_path, verbose=False, logger=None)
        
        # Load audio and detect beats
        y, sr = librosa.load(audio_path, sr=None)
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        self.beat_times = librosa.frames_to_time(beat_frames, sr=sr)
        
        # Cleanup
        if os.path.exists(audio_path):
            os.remove(audio_path)
        
        print(f"✅ Detected {len(self.beat_times)} beats at {tempo:.1f} BPM")
        return self.beat_times
    
    def auto_color_grade(self, style="cinematic"):
        """
        Apply AI-based color grading
        
        Args:
            style: Color grading style (cinematic, warm, cool, vintage)
        """
        print(f"🎨 Applying {style} color grading...")
        
        if style == "cinematic":
            # Increase contrast, slight warm tone
            self.video = self.video.fx(vfx.colorx, 1.2)
            self.video = self.video.fx(vfx.lum_contrast, contrast=0.3, contrast_thr=127)
        
        elif style == "warm":
            # Warm tones
            self.video = self.video.fx(vfx.colorx, 1.1)
        
        elif style == "cool":
            # Cool tones with high contrast
            self.video = self.video.fx(vfx.lum_contrast, contrast=0.2)
        
        elif style == "vintage":
            # Vintage film look
            self.video = self.video.fx(vfx.colorx, 0.9)
            self.video = self.video.fx(vfx.lum_contrast, contrast=-0.1)
        
        print("✅ Color grading applied")
        return self.video
    
    def create_beat_sync_edit(self, target_duration=None, clips_per_beat=1):
        """
        Create a music video style edit synced to beats
        
        Args:
            target_duration: Desired video length in seconds (None = use all scenes)
            clips_per_beat: Number of clips to show per beat
        """
        print("✂️ Creating beat-synchronized edit...")
        
        if not self.scenes:
            self.detect_scenes()
        if len(self.beat_times) == 0:
            self.detect_beats()
        
        edited_clips = []
        beat_idx = 0
        scene_idx = 0
        
        while beat_idx < len(self.beat_times) - 1:
            if target_duration and sum(c.duration for c in edited_clips) >= target_duration:
                break
            
            # Get beat interval
            beat_start = self.beat_times[beat_idx]
            beat_end = self.beat_times[beat_idx + 1]
            beat_duration = beat_end - beat_start
            
            # Select scene clip
            if scene_idx < len(self.scenes):
                scene_start, scene_end = self.scenes[scene_idx]
                
                # Extract clip from scene
                clip_start = scene_start + (scene_end - scene_start) * 0.3  # Start 30% into scene
                clip_end = min(clip_start + beat_duration, scene_end)
                
                if clip_end - clip_start > 0.1:
                    clip = self.video.subclip(clip_start, clip_end)
                    edited_clips.append(clip)
                
                scene_idx = (scene_idx + 1) % len(self.scenes)
            
            beat_idx += clips_per_beat
        
        # Concatenate clips
        final_video = concatenate_videoclips(edited_clips, method="compose")
        
        print(f"✅ Created {len(edited_clips)} synchronized clips ({final_video.duration:.1f}s)")
        return final_video
    
    
    def stabilize_video(self):
        """
        Apply basic video stabilization
        """
        print("📹 Stabilizing video...")
        # Basic stabilization using moviepy
        # Note: For advanced stabilization, use vidgear or other specialized libraries
        self.video = self.video.fx(vfx.resize, 0.95)  # Slight crop for stability margin
        print("✅ Video stabilized")
        return self.video
    
    def enhance_resolution(self, scale=1.5):
        """
        Upscale video resolution
        
        Args:
            scale: Scaling factor
        """
        print(f"🔍 Enhancing resolution by {scale}x...")
        self.video = self.video.fx(vfx.resize, scale)
        print("✅ Resolution enhanced")
        return self.video
    
    def export(self, custom_video=None):
        """
        Export the final edited video
        
        Args:
            custom_video: Optional custom VideoClip to export instead of self.video
        """
        print(f"💾 Exporting to {self.output_path}...")
        
        video_to_export = custom_video if custom_video else self.video
        
        video_to_export.write_videofile(
            self.output_path,
            codec='libx264',
            audio_codec='aac',
            fps=24,
            verbose=False,
            logger=None
        )
        
        print(f"✅ Video exported successfully to {self.output_path}")
        return self.output_path




**Final Function which will use the AIFilmEditor to create and export the final edited video**

In [ ]:
def main():
    """
    Example usage of the AI Film Editor
    """
    # Initialize editor
    editor = AIFilmEditor("input_video.mp4", "output_edited.mp4")
    
    # Create a beat-synchronized music video style edit
    print("\n=== Option 1: Beat-Sync Edit ===")
    editor.detect_scenes(threshold=25.0)
    editor.detect_beats(sensitivity=0.3)
    beat_sync_video = editor.create_beat_sync_edit(target_duration=120)
    editor.auto_color_grade(style="cinematic")
    editor.export(beat_sync_video)


if __name__ == "__main__":
    main()